## Block Timestamps

**[Johnnatan Messias](https://johnnatan-messias.github.io/), August 2024**

Understanding when each vote was cast is essential for analyzing the dynamics of governance in decentralized autonomous organizations (DAOs). However, event logs on-chain do not include timestamps by default. To obtain this information, we need to retrieve the block data associated with each transaction related to governance events.

Since querying an Ethereum Archive Node can be time-consuming, we use the [Nansen Query API](https://query.nansen.ai/) to fetch the block timestamps more efficiently.

Alternative data sources include platforms like [Dune](https://dune.com/) or Google BigQuery. If these services are unavailable, we've included a code snippet at the bottom of this Jupyter notebook that fetches block timestamps using an Ethereum Archive Node provided by Paradigm. You can also modify the code to use any other Ethereum node endpoints, including a node you operate yourself.

The notebook includes all the code used to fetch and store the block timestamps. The results are saved in a compressed CSV file:  
`./data/blocks/block_timestamp_9600000_20563001.csv.gz`


## Common variables and imports


In [ ]:
import pandas as pd

In [ ]:
import os
data_dir = os.path.realpath(os.path.join(
    os.getcwd(), "..", "data", "blocks"))
os.makedirs(data_dir, exist_ok=True)

## Using [Nansen Query API](https://query.nansen.ai/)


In [ ]:
import pandas_gbq
from google.cloud import bigquery
%load_ext google.cloud.bigquery


In [ ]:
progress_bar_type = 'tqdm_notebook'
project_id = 'Add your project id here'

In [ ]:
block_number_start, block_number_end = 9_600_000, 20_563_001

In [ ]:
sql = """
SELECT
    number, timestamp
FROM
    `nansen-query.raw_ethereum.blocks`
WHERE
    number >= {block_number_start} AND number <= {block_number_end}
ORDER BY
    number DESC
"""

In [ ]:
query = sql.format(block_number_start=block_number_start,
                   block_number_end=block_number_end)

df = pandas_gbq.read_gbq(
    query, project_id=project_id, progress_bar_type=progress_bar_type)

filename = "block_timestamp_{}_{}.csv.gz".format(
    block_number_start, block_number_end)

filedir = os.path.realpath(os.path.join(data_dir, filename))

df.to_csv(filedir, index=False, compression='gzip')

## Using Ethereum Archive Node


In [ ]:
from web3 import Web3
import requests as re

In [ ]:
block_number_start_test, block_number_end_test = 20_550_000, 20_563_001

In [ ]:
import sys
code_dir = os.path.realpath(os.path.join(os.getcwd(), "..", "src"))

sys.path.append(code_dir)

In [ ]:
from ethereum import get_blocks

In [ ]:
# This code connects to Paradigm Reth archive node (see https://x.com/gakonst/status/1702389827390546071)
eth_node = 'http://69.67.151.138:8545'

adapter = re.adapters.HTTPAdapter(pool_connections=20, pool_maxsize=20)
session = re.Session()
session.mount('http://', adapter)
session.mount('https://', adapter)

w3 = Web3(Web3.HTTPProvider(eth_node, session=session,
          request_kwargs={'timeout': 60}))

print("Is connected to Ethereum node: ", w3.is_connected())
print("The most recent block is: ", w3.eth.block_number)

In [ ]:
# Gathering blocks belonging to the range 20_550_000 and 20_563_001 for exemplification.
blocks = get_blocks(w3, block_numbers=range(
    block_number_start_test, block_number_end_test))
blocks_df = pd.DataFrame(blocks)
blocks_df = blocks_df[["number", "timestamp"]]
blocks_df.sort_values(by="number", inplace=True)
blocks_df.head()

In [ ]:
filename = "test_block_timestamp_{}_{}.csv.gz".format(
    block_number_start_test, block_number_end_test)

filedir = os.path.realpath(os.path.join(data_dir, filename))
blocks_df.to_csv(filedir, index=False, compression='gzip')